In [2]:
import pandas as pd
import numpy as np
import sys, urllib, re, json
import numba
from timeit import default_timer as timer
from numba import jit, njit
from numba import *
import string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from unidecode import unidecode
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import preprocessor as p
from  nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import pos_tag

#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)


stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)
 
# @numba.jit(nopython=True, forceobj=True)
def clean_EMOJI(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
 
    #after tweepy preprocessing the colon left remain after removing mentions
    #or RT sign in the beginning of the tweet
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
 
 
    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
 
    #filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
 
    #looping through conditions
    for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)
    
# @numba.jit(nopython=True, forceobj=True)   
def regular_process(text):
    text = str(text)
#     text = text.lower()
    text = re.sub(r"\+", "", text)
    text = re.sub(r",", "", text)
    text = re.sub(r"\.", "", text)
    text = re.sub(r"!", "", text)
    text = re.sub(r"\?", "", text)
    text = re.sub(r"'", "", text).strip()   
    text = re.sub(r":", "", text).strip()   
    text = re.sub(r"\s{2,}", " ", text).strip()   
    text = re.sub('RT','',text)
    text = " ".join(text.split())

    return text

# @numba.jit(nopython=True, forceobj=True)
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

# @numba.jit(nopython=True, forceobj=True)
def remove_URL(sample):
    
    """Remove URLs from a sample string"""
    res = ""
    for word in sample.split():
        if(word.find("http") != -1):
            word = ""
            res = res+word
        else:
            res = res+" "+word
        #res.join(" ")
    return res

# @numba.jit(nopython=True, forceobj=True)
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words.split():
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return " ".join(new_words)

# @numba.jit(nopython=True, forceobj=True)
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words.split():
        new_word = word.lower()
        new_words.append(new_word)
    return " ".join(new_words)

# @numba.jit(nopython=True, forceobj=True)
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words.split():
        if(word.find('#')!=-1 or word.find('@')!=-1):
            new_words.append(word)
        else:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
    return " ".join(new_words)

# @numba.jit(nopython=True, forceobj=True)
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words.split():
        if word.isdigit():
            new_word = p.number_to_words(word, andword="")
            new_words.append(new_word)
        else:
            new_words.append(word)
    return " ".join(new_words)

# @numba.jit(nopython=True, forceobj=True)
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words.split():
        if word not in stopwords.words('english'):
            new_words.append(word)
    return " ".join(new_words)

# @numba.jit(nopython=True, forceobj=True)
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return " ".join(stems)

def lemmas_words(sentence):

    filtered_sent = []
    filtered_sent_tag = {}
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))
    words = sentence.split()
#     print(words)
    ps = PorterStemmer()
#     print(pos_tag(words))
    for word, tag in pos_tag(words):
        if(word.find('#')!=-1 or word.find('@')!=-1):
            filtered_sent.append(word)
        else:
            wntag= tag[0].lower()
            wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
            lemma = lemmatizer.lemmatize(word, wntag) if wntag else word
            filtered_sent.append(lemma)
        
    return " ".join(filtered_sent)

def preprocess_by_stemming(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    text = re.sub('rt',' ',str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

def normalize(words):
    words = remove_non_ascii(words)
    words = regular_process(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    words = lemmas_words(words)
    return words

def preprocess(sample):
    sample = remove_URL(sample)
    sample = replace_contractions(sample)
    return normalize(sample)

In [3]:
# data = pd.read_csv('tweets_covid19_before_24_April_cleaned.csv')
# data

In [4]:
test5= 'RT @HenryMakow: FYI 30305 Sources s showing the #COVID19 / corona virus is over hyped, and likely fraud.\n\nhttps://t.co/FeyFqivla9'
# print(test5)
res = preprocess(test5)
res

'@henrymakow fyi thirty thousand, three hundred five source show #covid19 corona virus hype likely fraud'

In [2]:
import numba
import numpy as np
from timeit import default_timer as timer
from numba import jit, njit
# normal function to run on cpu 

def func(a):
    for i in range(10000000): 
        a[i]+= 1

# function optimized to run on gpu 
# @jit
# @cuda.jit
# @cuda.jit(device=True)

@jit
def func2(a): 
    for i in range(10000000): 
        a[i]+= 1
if __name__=="__main__": 
    n = 10000000
    a = np.ones(n, dtype = np.float64) 
    b = np.ones(n, dtype = np.float32) 
    
    start = timer() 
    func(a) 
    print("without GPU:", timer()-start)

    start = timer() 
    func2(a) 
    print("with GPU:", timer()-start) 

without GPU: 9.827770467964001
with GPU: 2.3705030330456793


In [6]:
# @numba.jit(nopython=True, forceobj=True)
def get_duplicate(AllTweets):
    result = pd.DataFrame()
    tweetChecklist = {}
    for current_tweet in AllTweets:
        # If tweet doesn't exist in the list
        if current_tweet not in tweetChecklist:
            j=0
            tweetChecklist[current_tweet]=j;
        else:
            tweetChecklist[current_tweet]= tweetChecklist[current_tweet]+1
#     print()
    result.append(tweetChecklist,ignore_index=True)
    return tweetChecklist

In [7]:
# duplicates = get_duplicate(all_tweets_covid19.original_text)
# duplicates

In [8]:
# tweets_Parkinsons="tweets_Parkinsons.csv"
# tweets_Epilepsy="tweets_Epilepsy.csv"
# tweets_HeartDisease="tweets_HeartDisease.csv"
# #tweets_Parkinson_ = pd.read_csv(tweets_Parkinsons)
# #tweets_Epilepsy_ = pd.read_csv(tweets_Epilepsy)
# #tweets_HeartDisease_ = pd.read_csv(tweets_HeartDisease)

In [9]:
tweets_epilepsy_1 = pd.read_csv('raw_Twitter_data/tweets_Parkinsons.csv')
tweets_Epilepsy_2 = pd.read_csv('tweets_Epilepsy_1MARS.csv')
tweets_epilepsy_1 = tweets_epilepsy_1.drop_duplicates().reset_index(drop=True)
tweets_Epilepsy_2= tweets_Epilepsy_2.drop_duplicates().reset_index(drop=True)
tweets_Epilepsy_3 = pd.read_csv('tweets_Epilepsy_1MARS.csv2')
tweets_Epilepsy_3= tweets_Epilepsy_3.drop_duplicates().reset_index(drop=True)
tweets_epilepsy = pd.read_csv('tweets_epilepsy.csv')
tweets_epilepsy = tweets_epilepsy.drop_duplicates().reset_index(drop=True)
tweets_epilepsy

,Unnamed: 0,Unnamed: 0.1,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries
0,0,0,1187341321901629441,Thu Oct 24 12:13:25 +0000 2019,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @AmEpilepsySoc: Terrific leadership team an...,@amepilepsysoc terrific leadership team great ...,"Sentiment(polarity=0.4, subjectivity=0.875)",0.4,0.875,en,0,2,OeGNeurologie,NaN,epilepsy,AmEpilepsySoc,Wien,NaN
1,1,1,1187341273490952193,Thu Oct 24 12:13:13 +0000 2019,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @AmEpilepsySoc: Have you used our interacti...,@amepilepsysoc used interactive program plan #...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,0,OeGNeurologie,NaN,AES2019,AmEpilepsySoc,Wien,NaN
2,2,2,1187340551567400960,Thu Oct 24 12:10:21 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",Any epilepsy nurses out there using T.E.C to g...,epilepsy nurses using tec gather data quolie 1...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,0,Mammylaine,NaN,NaN,NaN,NaN,NaN
3,3,3,1187340485616115712,Thu Oct 24 12:10:06 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",RT @YellaSunshine: My family suffered a huge l...,@yellasunshine family suffered huge loss passi...,"Sentiment(polarity=0.10625000000000004, subjec...",0.10625000000000004,0.7,en,0,2,Coolesssst,NaN,NaN,YellaSunshine,"Washington, DC",NaN
4,4,4,1187340465663815682,Thu Oct 24 12:10:01 +0000 2019,"<a href=""https://socialbee.io/"" rel=""nofollow""...",RT @aMundaneSarah: Every day can be a struggle...,@amundanesarah every day struggle disability #...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,8,aMundaneSarah,False,"epilepsy, AD",aMundaneSarah,"East Midlands, England",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8618,8618,8618,1234121903008288771,Sun Mar 01 14:22:45 +0000 2020,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @LotusOak2: #DidYouKnow that of the U.S. ch...,@lotusoak2 #didyouknow us children experience ...,"Sentiment(polarity=-0.05, subjectivity=0.0)",-0.05,0.0,en,0,14,xenonian1,NaN,"DidYouKnow, seizures, Epilepsy",LotusOak2,Roma Italy,NaN
8619,8619,8619,1234121012817485824,Sun Mar 01 14:19:12 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",RT @LotusOak2: #DidYouKnow that of the U.S. ch...,@lotusoak2 #didyouknow us children experience ...,"Sentiment(polarity=-0.05, subjectivity=0.0)",-0.05,0.0,en,0,14,goldwaterkid65,NaN,"DidYouKnow, seizures, Epilepsy",LotusOak2,United States,NaN
8620,8620,8620,1234120252600078336,Sun Mar 01 14:16:11 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",RT @LotusOak2: #DidYouKnow that of the U.S. ch...,@lotusoak2 #didyouknow us children experience ...,"Sentiment(polarity=-0.05, subjectivity=0.0)",-0.05,0.0,en,0,14,jjulz01,NaN,"DidYouKnow, seizures, Epilepsy",LotusOak2,"Tennessee, USA",NaN
8621,8621,8621,1234119519112716288,Sun Mar 01 14:13:16 +0000 2020,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @LotusOak2: #DidYouKnow that of the U.S. ch...,@lotusoak2 #didyouknow us children experience ...,"Sentiment(polarity=-0.05, subjectivity=0.0)",-0.05,0.0,en,0,14,qqqueenbeee,NaN,"DidYouKnow, seizures, Epilepsy",LotusOak2,TwilightZone🐸,NaN


In [10]:
frames = [tweets_epilepsy_1, tweets_Epilepsy_2, tweets_Epilepsy_3]
tweets_epilepsy= pd.concat(frames)
tweets_epilepsy = tweets_epilepsy.drop_duplicates().reset_index(drop=True)
len(tweets_epilepsy)

8623

In [11]:
tweets_epilepsy.to_csv('tweets_epilepsy.csv')

In [20]:
Tweets_covid19_after19MARCH = pd.read_csv("COVID19/tweets_covid19_after19MARCH.csv")
Tweets_covid19_after19MARCH = Tweets_covid19_after19MARCH.drop_duplicates().reset_index(drop=True)
print(len(Tweets_covid19_after19MARCH))
sentences = list(map(lambda x: preprocess(x),Tweets_covid19_after19MARCH.original_text))
Tweets_covid19_after19MARCH.original_text = sentences

249


In [21]:
frames = [tweets, Tweets_covid19_after19MARCH]
all_tweets_covid19 = pd.concat(frames)
all_tweets_covid19 = all_tweets_covid19.drop_duplicates().reset_index(drop=True)
all_tweets_covid19.to_csv('all_tweets_covid19.csv')

In [ ]:
tweets_epilepsy.original_text[0]

In [12]:
tweets_epilepsy

,Unnamed: 0,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries
0,0,1187341321901629441,Thu Oct 24 12:13:25 +0000 2019,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @AmEpilepsySoc: Terrific leadership team an...,RT AmEpilepsySoc Terrific leadership team grea...,"Sentiment(polarity=0.4, subjectivity=0.875)",0.4,0.875,en,0,2,OeGNeurologie,NaN,epilepsy,AmEpilepsySoc,Wien,NaN
1,1,1187341273490952193,Thu Oct 24 12:13:13 +0000 2019,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @AmEpilepsySoc: Have you used our interacti...,RT AmEpilepsySoc Have used interactive program...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,0,OeGNeurologie,NaN,AES2019,AmEpilepsySoc,Wien,NaN
2,2,1187340551567400960,Thu Oct 24 12:10:21 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",Any epilepsy nurses out there using T.E.C to g...,Any epilepsy nurses using T.E.C gather data QU...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,0,Mammylaine,NaN,NaN,NaN,NaN,NaN
3,3,1187340485616115712,Thu Oct 24 12:10:06 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",RT @YellaSunshine: My family suffered a huge l...,RT YellaSunshine My family suffered huge loss ...,"Sentiment(polarity=0.10625000000000004, subjec...",0.10625000000000004,0.7,en,0,2,Coolesssst,NaN,NaN,YellaSunshine,"Washington, DC",NaN
4,4,1187340465663815682,Thu Oct 24 12:10:01 +0000 2019,"<a href=""https://socialbee.io/"" rel=""nofollow""...",RT @aMundaneSarah: Every day can be a struggle...,RT aMundaneSarah Every day struggle disability...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.0,0.0,en,0,8,aMundaneSarah,False,"epilepsy, AD",aMundaneSarah,"East Midlands, England",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8618,8618,1234121903008288771,Sun Mar 01 14:22:45 +0000 2020,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @LotusOak2: #DidYouKnow that of the U.S. ch...,RT LotusOak2 DidYouKnow U.S. children experien...,"Sentiment(polarity=-0.05, subjectivity=0.0)",-0.05,0.0,en,0,14,xenonian1,NaN,"DidYouKnow, seizures, Epilepsy",LotusOak2,Roma Italy,NaN
8619,8619,1234121012817485824,Sun Mar 01 14:19:12 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",RT @LotusOak2: #DidYouKnow that of the U.S. ch...,RT LotusOak2 DidYouKnow U.S. children experien...,"Sentiment(polarity=-0.05, subjectivity=0.0)",-0.05,0.0,en,0,14,goldwaterkid65,NaN,"DidYouKnow, seizures, Epilepsy",LotusOak2,United States,NaN
8620,8620,1234120252600078336,Sun Mar 01 14:16:11 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",RT @LotusOak2: #DidYouKnow that of the U.S. ch...,RT LotusOak2 DidYouKnow U.S. children experien...,"Sentiment(polarity=-0.05, subjectivity=0.0)",-0.05,0.0,en,0,14,jjulz01,NaN,"DidYouKnow, seizures, Epilepsy",LotusOak2,"Tennessee, USA",NaN
8621,8621,1234119519112716288,Sun Mar 01 14:13:16 +0000 2020,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @LotusOak2: #DidYouKnow that of the U.S. ch...,RT LotusOak2 DidYouKnow U.S. children experien...,"Sentiment(polarity=-0.05, subjectivity=0.0)",-0.05,0.0,en,0,14,qqqueenbeee,NaN,"DidYouKnow, seizures, Epilepsy",LotusOak2,TwilightZone🐸,NaN


In [13]:
# data_cleaned  = tweets_epilepsy.clean_text
# data_cleaned
data  = tweets_epilepsy.original_text
data

0       RT @AmEpilepsySoc: Terrific leadership team an...
1       RT @AmEpilepsySoc: Have you used our interacti...
2       Any epilepsy nurses out there using T.E.C to g...
3       RT @YellaSunshine: My family suffered a huge l...
4       RT @aMundaneSarah: Every day can be a struggle...
                              ...                        
8618    RT @LotusOak2: #DidYouKnow that of the U.S. ch...
8619    RT @LotusOak2: #DidYouKnow that of the U.S. ch...
8620    RT @LotusOak2: #DidYouKnow that of the U.S. ch...
8621    RT @LotusOak2: #DidYouKnow that of the U.S. ch...
8622    #DidYouKnow that of the U.S. children who expe...
Name: original_text, Length: 8623, dtype: object

In [14]:
sample = "Blood test for Down's syndrome hailed http://bbc.in/1BO3eWQ"               
Best_sample = data[5]
best_sample2 = data[6]
best_sample3 = data[3]

In [15]:
best_sample2

'RT @EpiCARE_ERN: Sudden Unexpected Death in EPilepsy (#SUDEP) is a fatal complication of #epilepsy. \n\n#SUDEPActionDay2019 is a day to raise…'

In [16]:
print(tweets_epilepsy.original_text[5])
print(tweets_epilepsy.source[5])

RT @EpiCARE_ERN: #EpiCARE runs case discussion seminars for complex #epilepsy cases of #genetic, #metabolic, #infectious or unknown causes.…
<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>


In [17]:
data[2]

'Any epilepsy nurses out there using T.E.C to gather data on QUOLIE 10, NDDIE or HADS before a patient attends clini… https://t.co/utnqr6hWZc'

In [23]:
# clean_tweets(data[0])

'amepilepsysoc terrific leadership team great paners advancing epilepsy awareness treatment thank sharing chi…'

In [20]:
print(data[2])
print(remove_URL(data[2]))
text_to_word_list(data[2],unidecode)


Any epilepsy nurses out there using T.E.C to gather data on QUOLIE 10, NDDIE or HADS before a patient attends clini… https://t.co/utnqr6hWZc
 Any epilepsy nurses out there using T.E.C to gather data on QUOLIE 10, NDDIE or HADS before a patient attends clini…


'epilepsy nurses using tec gather data quolie 10 nddie hads patient attends clini…'

In [21]:
print(data[1])
Sentence1 = text_to_word_list(data[1], unidecode)
print(Sentence1)
#print(clean_tweets(data[1]))

RT @AmEpilepsySoc: Have you used our interactive program to plan your #AES2019 experience? Search by date, role, session type, and other he…
@amepilepsysoc used interactive program plan #aes2019 experience search date role session type he…


In [22]:
data = data.apply(lambda x: text_to_word_list(x, unidecode))
data.to_csv('epilepsy_texts.csv')

/home/h.grissette/.conda/envs/my_env_3.6/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [17]:
# data_cleaned.to_csv('parkinsons_cleaned_texts.csv')

/home/h.grissette/.conda/envs/my_env_3.6/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [26]:
file_model = data.copy()
file_model = file_model[data.str.len()>1]

In [25]:
# tweets_parkinson.clean_text[1]
tweets_epilepsy.clean_text = data
tweets_epilepsy.to_csv('tweets_epilepsy.csv')

In [27]:
sent = [row for row in file_model]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

INFO - 15:11:32: collecting all words and their counts
INFO - 15:11:32: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 15:11:32: collected 25892 word types from a corpus of 67835 words (unigram + bigrams) and 4729 sentences
INFO - 15:11:32: using 25892 counts as vocab in Phrases<0 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 15:11:32: source_vocab length 25892
INFO - 15:11:33: Phraser built with 7125 phrasegrams


['amepilepsysoc_have',
 'used_interactive',
 'program_plan',
 'aes2019_experience',
 'search_date',
 'role_session',
 'type_he']

In [28]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

INFO - 15:12:49: collecting all words and their counts
INFO - 15:12:49: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 15:12:49: collected 10534 word types from a corpus of 43095 raw words and 4729 sentences
INFO - 15:12:49: Loading a fresh vocabulary
INFO - 15:12:49: effective_min_count=3 retains 3167 unique words (30% of original 10534, drops 7367)
INFO - 15:12:49: effective_min_count=3 leaves 32957 word corpus (76% of original 43095, drops 10138)
INFO - 15:12:49: deleting the raw counts dictionary of 10534 items
INFO - 15:12:49: sample=1e-05 downsamples 3167 most-common words
INFO - 15:12:49: downsampling leaves estimated 5841 word corpus (17.7% of prior 32957)
INFO - 15:12:49: estimated required memory for 3167 words and 300 dimensions: 9184300 bytes
INFO - 15:12:49: resetting layer weights


Time to build vocab: 0.01 mins


In [32]:
w2v_model.init_sims(replace=True)
print(w2v_model.most_similar)

INFO - 15:14:31: precomputing L2-norms of word weight vectors


<bound method BaseWordEmbeddingsModel.most_similar of <gensim.models.word2vec.Word2Vec object at 0x2aab6ecbd2d0>>


In [38]:
w2v_model.most_similar

<bound method BaseWordEmbeddingsModel.most_similar of <gensim.models.word2vec.Word2Vec object at 0x2aab6ecbd2d0>>

In [33]:
w2v_model.save("word2vec.model")

INFO - 15:16:45: saving Word2Vec object under word2vec.model, separately None
INFO - 15:16:45: not storing attribute vectors_norm
INFO - 15:16:45: not storing attribute cum_table
INFO - 15:16:45: saved word2vec.model


In [34]:
file_export = file_model.copy()
file_export.to_csv('cleaned_dataset.csv', index=False)

/home/h.grissette/.conda/envs/my_env_3.6/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [39]:
from sklearn.cluster import KMeans

word_vectors = Word2Vec.load("word2vec.model").wv

model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors)

word_vectors.similar_by_vector(model.cluster_centers_[0], topn=10, restrict_vocab=None)

positive_cluster_center = model.cluster_centers_[0]
negative_cluster_center = model.cluster_centers_[1]

words = pd.DataFrame(word_vectors.vocab.keys())
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors.wv[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

words['cluster_value'] = [1 if i==0 else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

words.head(10)

words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

INFO - 15:48:46: loading Word2Vec object from word2vec.model
INFO - 15:48:46: loading wv recursively from word2vec.model.wv.* with mmap=None
INFO - 15:48:47: setting ignored attribute vectors_norm to None
INFO - 15:48:47: loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
INFO - 15:48:47: loading trainables recursively from word2vec.model.trainables.* with mmap=None
INFO - 15:48:47: setting ignored attribute cum_table to None
INFO - 15:48:47: loaded word2vec.model
INFO - 15:48:53: precomputing L2-norms of word weight vectors
/home/h.grissette/.conda/envs/my_env_3.6/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [40]:
sentiment_dictionary = pd.read_csv('sentiment_dictionary.csv')
sentiment_dictionary

,words,sentiment_coeff
0,great_paners,-1.005367
1,advancing,0.998064
2,epilepsy,-0.999886
3,awareness,1.001690
4,treatment,-1.004009
...,...,...
3162,browse_aicles,1.001180
3163,jcpnwemployers_are,-1.002699
3164,aware_consider,1.000008
3165,epilepsysociety_produced,1.000963
